In [ ]:
import torch
import numpy as np
import os

from models import DeepNetwork, DeepEnsemble
from utils import *
from train import *

print(torch.cuda.is_available())

In [ ]:
subset2col = {
        'flow+fund_mom+sentiment': list(range(56, 60)) + [47],
        'fund_ex_mom_flow': [59] + [x for x in range(46, 58) if x not in (list(range(54, 58)) + [47])],
        'stock': range(46),
        'fund': range(46, 59),
        'fund+sentiment': range(46, 60),
        'stock+fund': range(59),
        'F_r12_2+sentiment': [58, 59],
        'stock+sentiment': [59] + list(range(0, 46)),
        'stock+fund+sentiment': range(60),
        'F_r12_2+flow+sentiment': [47, 58, 59]
    }

In [ ]:
config = {
	'data_path': "./deep_learning/datasets/CharAll_na_rm_huge_train_variableall4_sentiment_full_new.npz",
	'split_lists_path': './deep_learning/sampling_folds/random_sampling_folds.npy',
	'subset': 'fund+sentiment',
	'num_layers': 1,
	'hidden_dim': [2**6],
	'dropout': 0.,
	'learning_rate': 0.001,
	'epochs': 512,
	'weighted_loss': False,
	'reg_l1': 0.0,
	'reg_l2': 0.001,
	'batch_size': 300000,
	'criteria': 'Factor_sharpe',
	'ensemble_members': 8,
	'random_seed': 15,
	'device': 'cuda' if torch.cuda.is_available() else 'cpu',
	'num_workers': 4
}

config['split_lists'] = np.load(config['split_lists_path'], allow_pickle = True)
config['input_dim'] = len(subset2col[config['subset']])

In [ ]:
model = DeepNetwork(config).to(device=config['device'])
print(model)

In [5]:
losses = {}
sharpes = {}
for subset in ['flow+fund_mom+sentiment', 'stock+sentiment', 'stock+fund+sentiment']:
	print('\nTRAINING FOR SUBSET: {}'.format(subset))
	config['subset'] = subset
	config['input_dim'] = len(subset2col[config['subset']])
	losses[subset], sharpes[subset] = run_one_subset(config)

Epoch 351 - Training Loss: 0.00037954, Val Loss: 0.00038083, Train Sharpe: 0.41472465, Validation Sharpe: 0.26323229
Epoch 401 - Training Loss: 0.00037970, Val Loss: 0.00038095, Train Sharpe: 0.41463464, Validation Sharpe: 0.28135699
Epoch 451 - Training Loss: 0.00037990, Val Loss: 0.00038106, Train Sharpe: 0.40640521, Validation Sharpe: 0.28535736
Epoch 464 - Training Loss: 0.00037998, Val Loss: 0.00038104, Train Sharpe: 0.39690080, Validation Sharpe: 0.28707445
Epoch 465 - Training Loss: 0.00037998, Val Loss: 0.00038104, Train Sharpe: 0.39777890, Validation Sharpe: 0.29122016
Epoch 466 - Training Loss: 0.00037999, Val Loss: 0.00038104, Train Sharpe: 0.39769647, Validation Sharpe: 0.28967088
Epoch 467 - Training Loss: 0.00037999, Val Loss: 0.00038104, Train Sharpe: 0.39856040, Validation Sharpe: 0.29324082
Epoch 468 - Training Loss: 0.00038000, Val Loss: 0.00038104, Train Sharpe: 0.39628172, Validation Sharpe: 0.29262388
Epoch 469 - Training Loss: 0.00038000, Val Loss: 0.00038104, Tra

In [6]:
print('test')

test


In [ ]:
losses = {}
sharpes = {}
for subset in ['stock', 'fund','F_r12_2+flow+sentiment', 'F_r12_2+sentiment']:
	print('\nTRAINING FOR SUBSET: {}'.format(subset))
	config['subset'] = subset
	config['input_dim'] = len(subset2col[config['subset']])
	losses[subset], sharpes[subset] = run_one_subset(config)

In [ ]:
crossval_loaders, masks = get_crossval_dataloaders(config['data_path'], config['split_lists'], config['subset'], batch_size=config['batch_size'])

avg_sharpe_fold_ens = 0.
for fold in range(len(crossval_loaders)):
	model_dirs = os.path.join('./Experiments/', config['subset'], 'fold' + str(fold+1))
	ensemble = DeepEnsemble(config, model_dirs)
	
	crossval_loaders, masks = get_crossval_dataloaders(config['data_path'], config['split_lists'], config['subset'], batch_size=config['batch_size'])
	_, _, test_loader = crossval_loaders[fold]['dataloaders']
	X_test, y_test = unload_data(test_loader)
	_, _, test_mask = masks[fold]
	
	ens_pred_test = ensemble.predict(X_test.float().to(config['device']))
	sharpe_test = evaluate_sharpe(ens_pred_test, y_test, test_mask)
	print(sharpe_test)
	avg_sharpe_fold_ens += sharpe_test

avg_sharpe_fold_ens /= len(crossval_loaders)
print(avg_sharpe_fold_ens)

In [ ]:
if __name__ == "__main__":
	losses, sharpes = run_all_subsets(config)
	print(losses)
	print(sharpes)